LANGSMITH_API_KEY : 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

TOGETHER_API_KEY : 2dde51635fbb84097198ddb134f5b04d580a467bc973e56ee9b340759336cb97



In [ ]:
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true" # 启用 LangChain V2 跟踪，可以在 https://smith.langchain.com/中查看跟踪数据
os.environ["TOGETHER_API_KEY"] = '2dde51635fbb84097198ddb134f5b04d580a467bc973e56ee9b340759336cb97' # 模型api key
os.environ["LANGSMITH_API_KEY"] = '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' # langsmith api key
# os.environ["OPENAI_API_KEY"] = ''


In [ ]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    base_url="https://api.together.xyz/v1",
    api_key=os.environ["TOGETHER_API_KEY"],
    model="mistralai/Mixtral-8x7B-Instruct-v0.1",
)

# -----------

In [19]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from English into Chinese"),
    HumanMessage(content="hello world!"),
]

result = model.invoke(messages)
result

AIMessage(content=' "hello world!" in Chinese is "你好，世界！" (nǐ hǎo, shìjiè!)\n\nThe Chinese language has many variations and dialects, but the most commonly used written form is Simplified Chinese, which is based on the Mandarin dialect. The characters I provided are Simplified Chinese characters.\n\nIn Traditional Chinese, which is used in Taiwan and Hong Kong, the same phrase would be written as "你好，世界！" (nǐ hǎo, shìjiè!) with the same pronunciation.\n\nI hope this helps! Let me know if you have any other questions.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 140, 'prompt_tokens': 21, 'total_tokens': 161, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'mistralai/Mixtral-8x7B-Instruct-v0.1', 'system_fingerprint': None, 'id': 'o4xfthb-62bZhn-96697392cc359e6b', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--416c9ce7-22f3-4288-8a81-240b3eacf620-0', usage_metadata={'input_tokens

# 结果解析方式
- StrOutputParser()：提取字符串
- JsonOutputParser()：解析 JSON 输出
- .content：获取原始输出内容


In [4]:
# 方式1 解析器
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

In [5]:
parser.invoke(result)

' "Hello World!" in Chinese is "你好，世界!" which is pronounced as "Nǐ hǎo, shìjiè!" in Mandarin. Here\'s the breakdown of the sentence:\n\n* "Nǐ" (你) means "you"\n* "Hǎo" (好) means "good" or "hello" in this context\n* "Shìjiè" (世界) means "world"\n\nSo, when you put it together, you get "Hello, World!" in Chinese.'

In [6]:
# 方式2 content属性获取结果str
result.content

' "Hello World!" in Chinese is "你好，世界!" which is pronounced as "Nǐ hǎo, shìjiè!" in Mandarin. Here\'s the breakdown of the sentence:\n\n* "Nǐ" (你) means "you"\n* "Hǎo" (好) means "good" or "hello" in this context\n* "Shìjiè" (世界) means "world"\n\nSo, when you put it together, you get "Hello, World!" in Chinese.'

In [ ]:
# 管道链接，将模型和解析器链接起来，逐个调用
chain = model | parser
print(chain.invoke(messages))

 "hello world!" in Chinese is "你好，世界！" (nǐ hǎo, shìjiè!)

Please note that the tone marks are not displayed correctly in some text editors or devices. The pinyin for the Chinese characters is provided to help with the pronunciation.


In [ ]:

# StrOutputParser()：提取字符串
# JsonOutputParser()：解析 JSON 输出
# PydanticOutputParser()：转换为结构化数据对象
# RetryWithErrorOutputParser()：支持错误重试解析

from langchain_core.output_parsers import StrOutputParser, JsonOutputParser, PydanticOutputParser

stroutparser, jsonoutparser = StrOutputParser(), JsonOutputParser()

print(result.content, end='\n')
print(stroutparser.invoke(result), end='\n')
print(jsonoutparser.invoke(result), end='\n')


# PromptTemplates 示例

In [15]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "将以下内容翻译为{language}，对话采用中文:"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)
prompt_template

ChatPromptTemplate(input_variables=['language', 'text'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language'], input_types={}, partial_variables={}, template='将以下内容翻译为{language}，对话采用中文:'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='{text}'), additional_kwargs={})])

In [16]:
prompt_result = prompt_template.format_prompt(text="你好，世界！", language="Japanese")
prompt_result

ChatPromptValue(messages=[SystemMessage(content='将以下内容翻译为Japanese，对话采用中文:', additional_kwargs={}, response_metadata={}), HumanMessage(content='你好，世界！', additional_kwargs={}, response_metadata={})])

In [17]:
prompt_result.to_messages()

[SystemMessage(content='将以下内容翻译为Japanese，对话采用中文:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='你好，世界！', additional_kwargs={}, response_metadata={})]

In [18]:
chain = prompt_template | model | parser
print(chain.invoke({'text': "你好，世界！", 'language' : "Japanese"}))

 こんにちは、sekai!

Note: In Japanese, "sekai" means "world", but it is a common and grammatically correct way to say "world" in this context in Japanese. The word "sekai" itself is often used in Japanese greetings, songs, and sayings, similar to how "world" is used in the English greeting "Hello, world!".


# 聊天机器人

In [21]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi! I'm Bob")]).content

" Hello Bob! It's nice to meet you. How can I assist you today? If you have any questions or need help with something, feel free to ask. I'm here to help!"

In [22]:
model.invoke([HumanMessage(content="What's my name?")]).content

" I'm unable to determine your name as I don't have any personal data about you, unless you're willing to share it with me. However, I'm here to provide information and answer questions to the best of my ability."

# Test

In [2]:
l = [
  "\r\n",
  "\r\n      \r\n          ",
  "\r\n      \r\n ",
  "\r\n      \r\n      发布于 2025年07月29日 08:42   更新于 2025年07月29日 17:06\r\n      \r\n      ",
  "\r\n      \r\n      \r\n      ",
  "\r\n      \r\n      ",
  "\r\n     "
]

In [3]:
pub_info = next((t.strip() for t in l if "发布于" in t), None)
pub_info

'发布于 2025年07月29日 08:42\xa0\xa0\xa0更新于 2025年07月29日 17:06'

In [11]:
import re
match = re.search(r"\d{4}年\d{2}月\d{2}日 \d{2}:\d{2}", pub_info or "")
match.group()

'2025年07月29日 08:42'

In [12]:
import re

# 提取发布时间、更新时间（如果有）
pub_time = None
update_time = None

match = re.search(r"发布于 (\d{4}年\d{2}月\d{2}日 \d{2}:\d{2})", pub_info or "")
if match:
    pub_time = match.group()

match = re.search(r"更新于 (\d{4}年\d{2}月\d{2}日 \d{2}:\d{2})", pub_info or "")
if match:
    update_time = match.group()

In [13]:
pub_time,update_time

('发布于 2025年07月29日 08:42', '更新于 2025年07月29日 17:06')

# Test

In [1]:
import re

In [30]:
s = '''
<h4><a href="https://china.caixin.com/2025-07-31/102347095.html">海南落马高官倪强被罢免全国人大代表</a>
'''
# s = s.strip()
pat = r'/\d{4}-\d{2}-\d{2}/\d+\.html'

In [32]:
res = re.search(pat, s)
print(res.group())

/2025-07-31/102347095.html


In [ ]:
import pandas as pd
pd.DataFrame([{'a': 1, 'b': 2}, {'a': 3, 'b': 4}])

In [2]:
import urllib.request
import os
from urllib.parse import urlparse

In [3]:
url = "https://ts1.tc.mm.bing.net/th/id/R-C.d2b3a4779fd2b9af70103d485bc8b664?rik=Xm7zutXMpsp91Q&riu=http%3a%2f%2fup.deskcity.org%2fpic_source%2fd2%2fb3%2fa4%2fd2b3a4779fd2b9af70103d485bc8b664.jpg&ehk=%2fh%2fipXq8Ihn81SbQdkphnzweLFLUGfD1%2fXncDcbLgRE%3d&risl=&pid=ImgRaw&r=0"

In [20]:
os.makedirs('tmp',exist_ok=True)

In [21]:
# 查看当前路径
print(os.getcwd())

D:\Codes\py\Agent


In [22]:
parsed_url = urlparse(url)
img_base_name = "img"
num = 0
filename = f"{img_base_name}_{num}.jpg"
num+=1

In [23]:
file_path = os.path.join('tmp', filename)
file_path

'tmp\\img_0.jpg'

In [24]:
req = urllib.request.Request(url)
with urllib.request.urlopen(req, timeout=30) as response:
    with open(file_path, 'wb') as f:
        f.write(response.read())

print(f"图片已保存到: {file_path}")

图片已保存到: tmp\img_0.jpg
